In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://media-exp3.licdn.com/dms/image/C4E1BAQEuTsJLViLkQw/company-background_10000/0/1587631889353?e=2159024400&v=beta&t=7N65emCWIoGvjrTaXUbHwLSaNEySNW0rAiEADaGOutI)br.linkedin.com

# **<span style="color:#DC143C;">Feyn is inspired by Richard Feynman's path integral formulation. That's why the python module to use it is called Feyn, and the Q in QLattice is for Quantum.</span>**

![](https://kwize.com/pics/Richard-Feynman-quote-about-honesty-from-Surely-You%27re-Joking%2C-Mr.-Feynman%21-1a14031.jpg)kwize.com

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 20px; background: #001f3f;"><i><b style="color:white;">Feyn and QLattice</b></i></h1></center>

"Feyn is a Python module for interacting with the QLattice."

"The QLattice is a machine learning technology that helps you search through an infinite list of potential mathematical models to solve your problem."

"It's a quantum-inspired simulation where you make decisions when exploring the data, giving you a good understanding of the relationships in your data and closing the loop between scientific inquiry and data science."

https://docs.abzu.ai/

https://docs.abzu.ai/docs/guides/getting_started/community.html

#Code by Casper Wilstrup https://www.kaggle.com/wilstrup/use-qlattice-to-predict-rainy-days-in-australia/notebook

In [ ]:
!pip install feyn

In [ ]:
import feyn

In [ ]:
df = pd.read_csv('../input/a-survey-on-obesity/obesity.csv', encoding='utf8')
pd.set_option('display.max_columns', None)
df.head(2)

In [ ]:
df.isnull().sum()

#When I ran the model I got :ValueError: {"error":"Spec 'What sort of baverages you take very often or on daily basis ?\n(Choose all options that apply ).' too long"}

#Let's rename what is too long?

In [ ]:
df = df.rename(columns={'ID': 'id', 'Completion time': 'completion', 'Email': 'email', 'Name': 'name', 'What is your gender ?': 'gender', 'Select your age group ?': 'age', 'Do you try to maintain balance diet ?': 'diet', 'How often do you eat junk food at your home or your workplace ?': 'junk', 'Select the weight category you belong to ?': 'category', 'Which of the following meals,you are most likely to have after busy day schedule ?Rate from 1-5 (with 1 being least likely and 5 being most).\n\nChapati,Dal,Rice, vegetables.': 'meal', 'Non- vegetarian meal.': 'vegetarian', 'Pizza,Burger,fries,cold drink.': 'pizza', 'How often you eat fruits and salad?': 'salad', 'What sort of baverages you take very often or on daily basis ?\n(Choose all options that apply ).':'beverages', 'How often you take alcoholic beverages ?': 'alcohol', 'Have you ever diagnosed with any of the conditions before :-\nDiabetes': 'diabetes', 'Hypothyroidism': 'hypothyroidism', 'PCOD (Polycystic ovarian disorder)': 'pcod', 'Are you aware that genes play a role in Obesity ?': 'genes', 'What do you think,which of the following cause Obesity?\n(Select all options that apply)': 'cause', 'Who among the following in your family ,suffer from similar condition?': 'who', 'How likely are you to spread Awareness that Genes play Important role in Obesity?': 'awareness'})

In [ ]:
df.head(2)

In [ ]:
#Code by Casper Wilstrup https://www.kaggle.com/wilstrup/use-qlattice-to-predict-rainy-days-in-australia/notebook

stypes = {
    "Start time": "cat",
    "completion": "cat",
    "email": "cat",
    "name": "cat",
    "gender": "cat",
    "age": "cat",
    "diet": "cat",
    "junk": "cat",
    "category": "cat",
    "How often you eat fruits and salad ?": "cat",
    "beverages": "cat",
    "alcohol": "cat",
    "diabetes": "cat",
    "hypothyroidism": "cat",
    "pcod": "cat",
    "cause": "cat",
    "who": "cat",
}

In [ ]:
#Code by Casper Wilstrup https://www.kaggle.com/wilstrup/use-qlattice-to-predict-rainy-days-in-australia/notebook

df = df.dropna(subset=["genes"])
df["genes"]=df["genes"]=="Yes"

In [ ]:
#Code by Bizen https://www.kaggle.com/hiro5299834/tps-apr-2021-deebtables/notebook

TARGET = 'genes' #Target could Not be float otherwise will result in valueError: Unknown label type: 'continuous'. Even after the encoding.

label_cols = ['Start time', 'completion', 'email', 'name', 'gender', 'age', 'diet', 'junk', 'category', 'How often you eat fruits and salad ?', 'alcohol', 'diabetes', 'hypothyroidism', 'pcod', 'cause', 'who']
numerical_cols = ['id', 'meal', 'vegetarian', 'pizza', 'awareness']

In [ ]:
#Code by Bizen https://www.kaggle.com/hiro5299834/tps-apr-2021-deebtables/notebook

from sklearn.preprocessing import LabelEncoder

def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

label_encoded_df = df[label_cols].apply(label_encoder)
numerical_df = df[numerical_cols]
target_df = df[TARGET]

df = pd.concat([label_encoded_df, numerical_df, target_df], axis=1)

#Check if the Label Encoding worked well.

In [ ]:
df.head()

In [ ]:
train, test = feyn.tools.split(df, ratio=(1,1), random_state=42)

In [ ]:
ql = feyn.connect_qlattice()

In [ ]:
ql.reset(random_seed=42)

In [ ]:
models = ql.auto_run(train, output_name="genes", kind="classification", stypes = stypes)

#The 1st time I got ValueError: {"error":"Spec 'What sort of baverages you take very often or on daily basis ?\n(Choose all options that apply ).' too long"}

#Hence I rename all the columns to avoid that issue.

In [ ]:
models[0].plot_roc_curve(train)
models[0].plot_roc_curve(test)

In [ ]:
models[0].plot_confusion_matrix(test, threshold=.3)

In [ ]:
models[0].plot_confusion_matrix(test, threshold=.5)

#The model is math!

What the QLattice actually finds is an equation that relates the input to the output. The user can control the compexity and structure of the equation in various ways. Here we just went with the defaults. Let's see the actual mathematical expression:

In [ ]:
models[0].sympify(2)

In [ ]:
models[0].plot(test)

![](https://i.pinimg.com/originals/b3/8c/15/b38c152b06f4fb26298b56983c107ded.jpg)pinterest.com

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thank you Casper Wilstrup @wilstrup for the script' )